In [ ]:
!pip install lime

In [ ]:
# ===============================
# LIME Explanation - Affairs Dataset
# ===============================

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from lime.lime_tabular import LimeTabularExplainer

# --- Step 1: Load dataset ---
df = pd.read_csv("Affairs.csv")

# Encode categorical variables if any
label_enc = LabelEncoder()
for col in df.select_dtypes(include="object").columns:
    df[col] = label_enc.fit_transform(df[col])

# Target variable (assuming 'affair' column = 1 if affair, else 0)
target = 'affairs' if 'affairs' in df.columns else df.columns[-1]
X = df.drop(columns=[target])
y = df[target]

# --- Step 2: Split and scale ---
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# --- Step 3: Train model ---
model = LogisticRegression()
model.fit(X_train_scaled, y_train)

# --- Step 4: LIME Explainer ---
explainer = LimeTabularExplainer(
    training_data=X_train_scaled,
    training_labels=y_train,
    feature_names=X.columns,
    class_names=['No Affair', 'Affair'],
    mode='classification'
)

# --- Step 5: Pick a sample to explain ---
i = 3  # any sample index
exp = explainer.explain_instance(
    data_row=X_test_scaled[i],
    predict_fn=model.predict_proba
)

# --- Step 6: Display results ---
print("\nLIME Explanation for Individual Record:")
print(f"Actual: {'Affair' if y_test.iloc[i]==1 else 'No Affair'}")
print(f"Predicted Probabilities: {model.predict_proba([X_test_scaled[i]])}")

exp.show_in_notebook(show_table=True)
# Or save explanation
exp.save_to_file('lime_affair.html')
print("LIME explanation saved as lime_affair.html")
